<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment5_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import torch
tfds.disable_progress_bar()

In [2]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')



# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


# Loading the dataset

In [3]:
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV4CJRR/imdb_reviews-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV4CJRR/imdb_reviews-test.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV4CJRR/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [4]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [5]:
# Shuffling the dataset
buffer_size = 10000
batch_size = 64
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Building the model

## Representing the  text

In [6]:
vocab_size = 10000
encoder = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
encoder.adapt(train_dataset.map(lambda x,y: x))

# Store vocabulary
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U17')

## Building the LSTM model

### With different embedding sizes

In [7]:
# Creating the model
model = tf.keras.Sequential([encoder,
                             tf.keras.layers.Embedding(
                                 input_dim = len(encoder.get_vocabulary()),
                                 output_dim = 64,
                                 mask_zero = True),
                            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                            tf.keras.layers.Dense(64, activation = 'relu'),
                            tf.keras.layers.Dense(1, activation = 'sigmoid')])


# Compile the model for training
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics = ['accuracy'])



In [8]:
# Training the model
history = model.fit (train_dataset,
                     epochs = 10, 
                     validation_data = test_dataset,
                     validation_steps = 30)




test_loss, test_acc = model.evaluate (test_dataset)
print('Test loss: ', test_loss)
print('Test acurracy: ', test_acc)



Epoch 1/10
391/391 [==============================] - 113s 255ms/step - loss: 0.6533 - accuracy: 0.5838 - val_loss: 0.5839 - val_accuracy: 0.7047
Epoch 2/10
391/391 [==============================] - 94s 238ms/step - loss: 0.4315 - accuracy: 0.7960 - val_loss: 0.4267 - val_accuracy: 0.8406
Epoch 3/10
391/391 [==============================] - 91s 232ms/step - loss: 0.3234 - accuracy: 0.8577 - val_loss: 0.3805 - val_accuracy: 0.8370
Epoch 4/10
391/391 [==============================] - 91s 232ms/step - loss: 0.2594 - accuracy: 0.8930 - val_loss: 0.3254 - val_accuracy: 0.8641
Epoch 5/10
391/391 [==============================] - 91s 231ms/step - loss: 0.2261 - accuracy: 0.9104 - val_loss: 0.3112 - val_accuracy: 0.8750
Epoch 6/10
391/391 [==============================] - 93s 235ms/step - loss: 0.2067 - accuracy: 0.9249 - val_loss: 0.3287 - val_accuracy: 0.8781
Epoch 7/10
391/391 [==============================] - 92s 234ms/step - loss: 0.1787 - accuracy: 0.9341 - val_loss: 0.3212 - val_a

### With an average of all hidden states to fully connected layer

In [ ]:
# Creating the model
model = tf.keras.Sequential([encoder,
                             tf.keras.layers.Embedding(
                                 input_dim = len(encoder.get_vocabulary()),
                                 output_dim = 64,
                                 mask_zero = True),
                            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)),
                            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)),
                            tf.keras.layers.Dense(64, activation = 'relu'),
                            tf.keras.layers.Dense(1, activation = 'sigmoid')])


# Compile the model for training
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics = ['accuracy'])

# Training the model
history = model.fit (train_dataset,
                     epochs = 10, 
                     validation_data = test_dataset,
                     validation_steps = 30)




test_loss, test_acc = model.evaluate (test_dataset)
print('Test loss: ', test_loss)
print('Test acurracy: ', test_acc)
